In [53]:
import json
import pandas as pd


def is_feature_documented(ftname):
    df = pd.read_csv('../optionsAndImportance.csv', header=None)
    return ftname in df[0].values

def get_experts_voice(ftname):
    if (ftname == 'nbyes'):
        return "see discussions with E2 and simply common-sense knowlegde about tristate options and the way Linux kernel works"
    df_experts = pd.read_csv('LinuxSizeExpertsVoice-E1E2.csv')
    df_experts.rename(columns={'Option name' : 'option'}, inplace=True)
    if ftname in df_experts['option'].values:
        return df_experts.query("option == '" + ftname + "'")['Impact on kernel size binaries?'].values[0]
    else:
        return None

assert(is_feature_documented('UBSAN_SANITIZE_ALL'))

print(get_experts_voice("X86_NEED_RELOCS"))

def get_groups(gname):
    gnumber = gname[6:] # group_ is 5 chars
    with open('covariance_groups.json') as json_file:
        groups = json.load(json_file)
        if gnumber in groups:
            return groups[gnumber]
        else:
            return None

top = 50
i = 0

annotated_features = pd.DataFrame(columns=['rank', 'ftname', 'group', 'documented?', 'experts_voice'])
with open('l_features_by_mean_ranking.json') as json_file:
    fts = json.load(json_file)
    for ftname in fts:
        if i >= top:
            break
        if ("group_" in ftname):
            fgroup = get_groups(ftname)
            assert(fgroup is not None)
            # at least one is documented in the fgroup? 
            documented = False
            for ft in fgroup:
                if (is_feature_documented(ft)):
                    documented = True
                    break
            for ft in fgroup:
                annotated_features.loc[i] = (i, ft, ftname, is_feature_documented(ft), get_experts_voice(ft))
                i = i + 1
        elif "DEBUG" in ftname:
            annotated_features.loc[i] = (i, ftname, None, is_feature_documented(ftname), get_experts_voice(ftname))
            i = i + 1
        #elif "nbyes" in ftname:
        #    annotated_features.loc[i] = (i, ftname, None, is_feature_documented(ftname), None)
        #    i = i + 1
        else:
            annotated_features.loc[i] = (i, ftname, None, is_feature_documented(ftname), get_experts_voice(ftname))
            i = i + 1

don't know


In [54]:
annotated_features['experts_or_documented'] = (annotated_features['documented?'] == True) | (pd.isna(annotated_features['experts_voice']) == False)
annotated_features

,rank,ftname,group,documented?,experts_voice,experts_or_documented
0,0,DEBUG_INFO,None,True,None,True
1,1,nbyes,None,False,see discussions with E2 and simply common-sens...,True
2,2,X86_NEED_RELOCS,group_129,False,don't know,True
3,3,RANDOMIZE_BASE,group_129,False,This shouldn't affect the actual kernel code s...,True
4,4,DEBUG_INFO_REDUCED,None,True,None,True
5,5,DEBUG_INFO_SPLIT,None,True,None,True
6,6,UBSAN_SANITIZE_ALL,None,True,None,True
7,7,KASAN,group_43,False,yes. This is a debugging feature. Adds guard ...,True
8,8,KASAN_OUTLINE,group_43,True,None,True
9,9,UBSAN_ALIGNMENT,None,False,don't know,True


In [55]:
len(annotated_features.query("experts_or_documented == True"))

44

In [56]:
#  only 7 options are documented as having a clear influence on size; 
len(annotated_features[annotated_features['documented?'] == True])

11

In [9]:
annotated_features.query("experts_or_documented == False")
# group 153: Disable this option to remove the block layer support from the kernel. This may be useful for embedded devices.
# DRM_* are obviously options that increase size when not compiled as module 
# eg Choose this option if you have an ATI Radeon graphics card. There are both PCI and AGP versions. You don't need to choose this to run the Radeon in plain VGA mode.
# https://cateee.net/lkddb/web-lkddb/RANDOMIZE_MEMORY.html: size not explicitly mentioned but has effect on memory management

,rank,ftname,group,documented?,experts_voice,experts_or_documented
16,16,DRM_AMDGPU,None,False,None,False
19,19,DRM_RADEON,None,False,None,False
21,21,ATH5K_TRACER,None,False,None,False
24,24,DRM_I915,None,False,None,False
29,29,DRM_VMWGFX,None,False,None,False
30,30,IOSCHED_NOOP,group_153,False,None,False
31,31,SBITMAP,group_153,False,None,False
32,32,BLOCK,group_153,False,None,False
33,33,DRM_QXL,None,False,None,False
34,34,MISDN_W6692,None,False,None,False


In [48]:
import numpy as np
with open('covariance_groups.json') as json_file:
    groups = json.load(json_file)
    print(len(groups))
    freq_groups = []
    
    for group in groups:
        freq_groups.append(len(groups[group]))                    
        #with open('fli_covariance.json') as json_file:
        #    imp_groups = json.load(json_file)
        #    print(groups[group])
        #    if groups[group] in imp_groups:
        #        print("implications!")
        # with open('fli_no_covariance.json') as json_file:
        #    excl_groups = json.load(json_file)
        #    print(groups[group])
        #    if groups[group] in excl_groups:
        #        print("exclusions!")

print("The number of groups, in total, is ", len(freq_groups)) 
print("The number of features that are involved in a group is, in total, ", np.sum(freq_groups)) 
print("The vast majority of groups has either two features ", round((173/234)*100, 1), "or three features", round((34/234)*100, 1))
print("The largest group has 42 features.")

np.mean(freq_groups), np.min(freq_groups), np.max(freq_groups), np.median(freq_groups), np.unique(freq_groups, return_counts=True)

234
The number of groups, in total, is  234
The number of features that are involved in a group is, in total,  674
The vast majority of groups has either two features  73.9 or three features 14.5
The largest group has 42 features.


(2.8803418803418803,
 2,
 42,
 2.0,
 (array([ 2,  3,  4,  5,  6,  7,  8,  9, 11, 13, 16, 17, 19, 42]),
  array([172,  34,  14,   2,   1,   3,   1,   1,   1,   1,   1,   1,   1,
           1])))

In [41]:
len(imp_groups)

500